# Testing out different Vectorizers and regressors

## Vectorizers

    1. tf-idf vs tf vs count
    2. max_features?
    3. stop_words?
    4. ngrams?

Feature importance?


In [7]:
from comet_ml import Experiment
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC

In [8]:
df = pd.read_csv('../data/tweets_50.csv')
X = df['text_tokenized']
y = df['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [9]:
pipe = Pipeline([
    ('vect', TfidfVectorizer()),
    ('model', LinearSVC())
])

search_grid = {
    # 'vect': [TfidfVectorizer(), TfidfVectorizer(use_idf=False), CountVectorizer()],
    # 'vect__strip_accents': ['ascii'],
    # 'vect__max_features': [10_000, 15_000, 20_000, 25_000, 30_000, 40_000, 50_000],
    # 'vect__max_features': [10_000, 20_000, 30_000],
    # 'vect__stop_words': ['english', None],
    # 'vect__ngram_range': [(1, 1), (2, 2), (3, 3)]
}
grid = GridSearchCV(pipe, search_grid, n_jobs=6, verbose=2,
                    scoring='accuracy')

grid.fit(X_train, y_train)
results_cv = pd.DataFrame(grid.cv_results_)

Fitting 5 folds for each of 1 candidates, totalling 5 fits


In [10]:
print(grid.best_params_)
print(grid.best_score_)

clf = grid.best_estimator_.named_steps['model']
vect = grid.best_estimator_.named_steps['vect']
# zipped = zip(vect.get_feature_names(), reg.feature_importances_)
# feature_importance = pd.DataFrame(zipped, columns=["feature", "value"])

# feature_importance = feature_importance.sort_values('value', ascending=False)

# sns.barplot(data=feature_importance[:30], y='feature', x='value')
# results_cv.sort_values('rank_test_score')
# print(results_cv.T.to_markdown())
# mean_test_cols = results_cv.columns[results_cv.columns.map(lambda s: s.startswith('mean_test_'))]
# print(results_cv[['param_vect', *mean_test_cols]].T.to_markdown())

{}
0.9917123597766487


In [12]:
for i in range(len(grid.cv_results_['params'])):

    exp = Experiment(workspace="henrystoll",
                     project_name="nlp-token-sklearn",
                     api_key="HeH9EtfDC2KUlCOjeQaU1CuOM",)
    for k,v in grid.cv_results_.items():
        if k == "params":
            exp.log_parameters(v[i])
        else:
            exp.log_metric(k,v[i])
    exp.end()

# results_cv.head()
results_cv[['mean_fit_time', 'params', 'mean_test_score']]

COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.ml/henrystoll/nlp-token-sklearn/baee1d6615f6455eb9c62655eec83821
COMET INFO:   Uploads:
COMET INFO:     environment details      : 1
COMET INFO:     filename                 : 1
COMET INFO:     git metadata             : 1
COMET INFO:     git-patch (uncompressed) : 1 (240 KB)
COMET INFO:     installed packages       : 1
COMET INFO: ---------------------------
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.ml https://www.comet.ml/henrystoll/nlp-token-sklearn/96f6b72c6f214668a8077ab007903e0e

COMET WARNING: Empty mapping given to log_params({}); ignoring
COMET INFO: -----------------------

,mean_fit_time,params,mean_test_score
0,71.87791,{},0.991712
